In [1]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [2]:
# read in with pandas
winter1 = pd.read_csv ('elm20-21data.txt')
winter2 = pd.read_csv ('elm21-22data.txt')
winter3 = pd.read_csv ('elm22-23data.txt')

In [3]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [4]:
# mask out days without precipitation
codes = ['GS','IC','PL','SG','SN']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [5]:
precip_days2

,station,datetime,precip(mm),wxcodes
3,ELM,2021-12-01 01:53:00,T,-SN
29,ELM,2021-12-02 02:53:00,T,-SN
164,ELM,2021-12-07 03:59:00,T,-SN
165,ELM,2021-12-07 04:07:00,T,-SN
166,ELM,2021-12-07 04:09:00,T,-SN
...,...,...,...,...
2780,ELM,2022-02-25 05:36:00,T,-SN
2785,ELM,2022-02-25 08:06:00,0.76,-SN BR
2786,ELM,2022-02-25 08:17:00,1.52,-SN BR
2787,ELM,2022-02-25 08:38:00,2.54,-SN BR


In [6]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [7]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [8]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [9]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [10]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [11]:
elm = LocationInfo(
    'Elmira', 'New York', 
    latitude= 42.0898,
    longitude=  -76.8077, 
    timezone='America/New_York'
)

elm

LocationInfo(name='Elmira', region='New York', timezone='America/New_York', latitude=42.0898, longitude=-76.8077)

In [12]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(elm.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [13]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(elm.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [14]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(elm.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [15]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:16:46  2020-12-01 16:36:01  2020-12-01
1   2020-12-02 07:17:48  2020-12-02 16:35:46  2020-12-02
2   2020-12-03 07:18:49  2020-12-03 16:35:33  2020-12-03
3   2020-12-04 07:19:49  2020-12-04 16:35:23  2020-12-04
4   2020-12-05 07:20:48  2020-12-05 16:35:15  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:49:58  2021-02-24 17:51:22  2021-02-24
86  2021-02-25 06:48:26  2021-02-25 17:52:35  2021-02-25
87  2021-02-26 06:46:53  2021-02-26 17:53:48  2021-02-26
88  2021-02-27 06:45:19  2021-02-27 17:55:01  2021-02-27
89  2021-02-28 06:43:44  2021-02-28 17:56:14  2021-02-28

[90 rows x 3 columns]


In [16]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:16:30  2021-12-01 16:36:04  2021-12-01
1   2021-12-02 07:17:33  2021-12-02 16:35:49  2021-12-02
2   2021-12-03 07:18:34  2021-12-03 16:35:35  2021-12-03
3   2021-12-04 07:19:35  2021-12-04 16:35:25  2021-12-04
4   2021-12-05 07:20:34  2021-12-05 16:35:16  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:50:21  2022-02-24 17:51:04  2022-02-24
86  2022-02-25 06:48:48  2022-02-25 17:52:17  2022-02-25
87  2022-02-26 06:47:15  2022-02-26 17:53:30  2022-02-26
88  2022-02-27 06:45:41  2022-02-27 17:54:43  2022-02-27
89  2022-02-28 06:44:07  2022-02-28 17:55:56  2022-02-28

[90 rows x 3 columns]


In [17]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:16:15  2022-12-01 16:36:08  2022-12-01
1   2022-12-02 07:17:18  2022-12-02 16:35:52  2022-12-02
2   2022-12-03 07:18:20  2022-12-03 16:35:38  2022-12-03
3   2022-12-04 07:19:20  2022-12-04 16:35:27  2022-12-04
4   2022-12-05 07:20:19  2022-12-05 16:35:18  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:50:43  2023-02-24 17:50:46  2023-02-24
86  2023-02-25 06:49:11  2023-02-25 17:51:59  2023-02-25
87  2023-02-26 06:47:38  2023-02-26 17:53:13  2023-02-26
88  2023-02-27 06:46:04  2023-02-27 17:54:26  2023-02-27
89  2023-02-28 06:44:30  2023-02-28 17:55:38  2023-02-28

[90 rows x 3 columns]


In [18]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [19]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ELM,2020-12-01 17:53:00,0.5100,-SN BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
1,ELM,2020-12-01 22:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
2,ELM,2020-12-02 00:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
3,ELM,2020-12-02 05:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
4,ELM,2020-12-02 06:53:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
...,...,...,...,...,...,...,...
272,ELM,2021-02-22 16:53:00,0.0001,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54
273,ELM,2021-02-22 22:53:00,0.0001,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54
274,ELM,2021-02-23 14:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08
275,ELM,2021-02-23 15:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08


In [20]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ELM,2021-12-01 01:53:00,0.0001,-SN,2021-12-01,2021-12-01 07:16:30,2021-12-01 16:36:04
1,ELM,2021-12-02 02:53:00,0.0001,-SN,2021-12-02,2021-12-02 07:17:33,2021-12-02 16:35:49
2,ELM,2021-12-08 10:53:00,0.0001,-SN,2021-12-08,2021-12-08 07:23:22,2021-12-08 16:35:05
3,ELM,2021-12-08 11:53:00,0.0001,-SN BR,2021-12-08,2021-12-08 07:23:22,2021-12-08 16:35:05
4,ELM,2021-12-08 19:53:00,0.0001,-SN,2021-12-08,2021-12-08 07:23:22,2021-12-08 16:35:05
...,...,...,...,...,...,...,...
132,ELM,2022-02-18 14:53:00,0.0001,-SN,2022-02-18,2022-02-18 06:59:14,2022-02-18 17:43:36
133,ELM,2022-02-18 15:53:00,0.0001,-SN,2022-02-18,2022-02-18 06:59:14,2022-02-18 17:43:36
134,ELM,2022-02-19 12:53:00,0.0001,-SN,2022-02-19,2022-02-19 06:57:48,2022-02-19 17:44:51
135,ELM,2022-02-19 13:53:00,0.0001,-SN,2022-02-19,2022-02-19 06:57:48,2022-02-19 17:44:51


In [21]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ELM,2022-12-11 12:53:00,0.2500,-SN BR,2022-12-11,2022-12-11 07:25:45,2022-12-11 16:35:14
1,ELM,2022-12-11 13:53:00,1.5200,+SN FG,2022-12-11,2022-12-11 07:25:45,2022-12-11 16:35:14
2,ELM,2022-12-11 14:53:00,1.5200,SN FG,2022-12-11,2022-12-11 07:25:45,2022-12-11 16:35:14
3,ELM,2022-12-11 15:53:00,0.7600,-SN BR,2022-12-11,2022-12-11 07:25:45,2022-12-11 16:35:14
4,ELM,2022-12-15 14:53:00,0.7600,-SN,2022-12-15,2022-12-15 07:28:50,2022-12-15 16:36:00
...,...,...,...,...,...,...,...
153,ELM,2023-02-25 16:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:49:11,2023-02-25 17:51:59
154,ELM,2023-02-25 17:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:49:11,2023-02-25 17:51:59
155,ELM,2023-02-25 23:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:49:11,2023-02-25 17:51:59
156,ELM,2023-02-27 22:53:00,1.0200,-SN BR,2023-02-27,2023-02-27 06:46:04,2023-02-27 17:54:26


In [22]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
5,ELM,2020-12-02 07:53:00,0.2500,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
6,ELM,2020-12-02 08:53:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
7,ELM,2020-12-02 11:53:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
8,ELM,2020-12-02 12:53:00,0.2500,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
9,ELM,2020-12-02 13:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
...,...,...,...,...,...,...,...
271,ELM,2021-02-22 15:53:00,0.5100,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54
272,ELM,2021-02-22 16:53:00,0.0001,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54
274,ELM,2021-02-23 14:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08
275,ELM,2021-02-23 15:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08


In [23]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [24]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
6,ELM,2020-12-02 08:53:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46,2020-12-02 08:17:48,2020-12-02 15:35:46
7,ELM,2020-12-02 11:53:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46,2020-12-02 08:17:48,2020-12-02 15:35:46
8,ELM,2020-12-02 12:53:00,0.2500,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46,2020-12-02 08:17:48,2020-12-02 15:35:46
9,ELM,2020-12-02 13:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46,2020-12-02 08:17:48,2020-12-02 15:35:46
17,ELM,2020-12-09 11:53:00,0.0001,-SN,2020-12-09,2020-12-09 07:24:28,2020-12-09 16:35:08,2020-12-09 08:24:28,2020-12-09 15:35:08
...,...,...,...,...,...,...,...,...,...
270,ELM,2021-02-22 14:53:00,0.7600,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54,2021-02-22 07:53:01,2021-02-22 16:48:54
271,ELM,2021-02-22 15:53:00,0.5100,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54,2021-02-22 07:53:01,2021-02-22 16:48:54
274,ELM,2021-02-23 14:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08,2021-02-23 07:51:30,2021-02-23 16:50:08
275,ELM,2021-02-23 15:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08,2021-02-23 07:51:30,2021-02-23 16:50:08


In [25]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [26]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [27]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.1977872727272728
2021-2022: 0.24858297872340424
2022-2023: 0.2678


In [28]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ELM,2020-12-01 17:53:00,0.5100,-SN BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
1,ELM,2020-12-01 22:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
2,ELM,2020-12-02 00:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
3,ELM,2020-12-02 05:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
4,ELM,2020-12-02 06:53:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
...,...,...,...,...,...,...,...
263,ELM,2021-02-21 03:53:00,0.0001,-SN,2021-02-21,2021-02-21 06:54:30,2021-02-21 17:47:39
264,ELM,2021-02-21 04:53:00,0.0001,-SN,2021-02-21,2021-02-21 06:54:30,2021-02-21 17:47:39
265,ELM,2021-02-21 05:53:00,0.0001,-SN,2021-02-21,2021-02-21 06:54:30,2021-02-21 17:47:39
266,ELM,2021-02-21 06:53:00,0.0001,-SN,2021-02-21,2021-02-21 06:54:30,2021-02-21 17:47:39


In [29]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25552\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [30]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [31]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [32]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.2578576271186441
2021-2022: 0.10563333333333334
2022-2023: 0.4325545454545454


In [33]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('SO.elm.csv', index=False)